<a href="https://colab.research.google.com/github/feaviolp/msc-project/blob/main/NIJ%20statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(haven)
library(dplyr)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [29]:

NIJ <- read.csv("https://raw.githubusercontent.com/feaviolp/msc-project/main/NIJ_s_Recidivism_Challenge_Full_Dataset_20240222.csv")
View(NIJ)

ID,Gender,Race,Age_at_Release,Residence_PUMA,Gang_Affiliated,Supervision_Risk_Score_First,Supervision_Level_First,Education_Level,Dependents,⋯,DrugTests_Meth_Positive,DrugTests_Other_Positive,Percent_Days_Employed,Jobs_Per_Year,Employment_Exempt,Recidivism_Within_3years,Recidivism_Arrest_Year1,Recidivism_Arrest_Year2,Recidivism_Arrest_Year3,Training_Sample
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,M,BLACK,43-47,16,false,3,Standard,At least some college,3 or more,⋯,0.00000000,0.000000000,0.4885621,0.4476103,false,false,false,false,false,1
2,M,BLACK,33-37,16,false,6,Specialized,Less than HS diploma,1,⋯,0.00000000,0.000000000,0.4252336,2.0000000,false,true,false,false,true,1
3,M,BLACK,48 or older,24,false,7,High,At least some college,3 or more,⋯,0.16666667,0.000000000,0.0000000,0.0000000,false,true,false,true,false,1
4,M,WHITE,38-42,16,false,7,High,Less than HS diploma,1,⋯,0.00000000,0.000000000,1.0000000,0.7189961,false,false,false,false,false,1
5,M,WHITE,33-37,16,false,4,Specialized,Less than HS diploma,3 or more,⋯,0.05882353,0.000000000,0.2035623,0.9293893,false,true,true,false,false,1
6,M,WHITE,38-42,17,false,5,Standard,High School Diploma,0,⋯,0.00000000,0.000000000,0.6742520,0.3078382,false,false,false,false,false,0
7,M,BLACK,48 or older,18,false,2,Standard,Less than HS diploma,2,⋯,0.00000000,0.000000000,0.0000000,0.0000000,false,true,false,false,true,1
8,M,BLACK,38-42,16,false,5,High,High School Diploma,3 or more,⋯,0.00000000,0.000000000,0.4694464,0.5251618,false,false,false,false,false,0
9,F,BLACK,43-47,5,,7,High,High School Diploma,0,⋯,NA,NA,0.0000000,0.0000000,true,true,true,false,false,1


Is there a statisitically significant difference in prior arrest episodes and PP violation charges betweeen black males and white males?

In [15]:
shapiro.test(NIJ$Race)

ERROR: Error in shapiro.test(NIJ$Race): is.numeric(x) is not TRUE


In [30]:
NIJ$Race <-  as.integer(factor(NIJ$Race))



In [31]:
NIJ$Prior_Arrest_Episodes_PPViolationCharges <- replace(NIJ$Prior_Arrest_Episodes_PPViolationCharges, NIJ$Prior_Arrest_Episodes_PPViolationCharges == '5 or more', '5')
NIJ$Prior_Arrest_Episodes_PPViolationCharges <-  as.integer(NIJ$Prior_Arrest_Episodes_PPViolationCharges)
View(NIJ$Prior_Arrest_Episodes_PPViolationCharges)

[1] 4 5 5 3 3 0 5 5 3 5 3 3 1 5 4 4 4 5 5 5 3 5 1 5 5 5 5 5 2 1 5 4 5 5 5 4
   [37] 5 5 5 0 5 4 5 5 5 3 4 0 1 0 5 5 4 5 5 2 5 5 5 4 3 3 5 5 5 2 5 2 5 5 3 5
   [73] 5 1 5 2 4 3 5 1 5 5 5 5 5 5 1 5 5 4 3 5 2 5 1 5 5 3 3 5 3 2 1 5 2 5 5 0
  [109] 5 5 5 1 5 5 5 5 4 5 5 2 5 5 5 3 5 2 3 0 5 5 2 5 3 5 2 5 5 5 5 5 4 5 5 5
  [145] 2 5 0 5 4 5 5 1 5 5 5 5 5 5 2 0 5 1 4 4 1 5 5 3 5 3 3 5 5 4 5 5 5 1 3 5
  [181] 5 5 4 5 5 5 4 5 5 4 5 5 4 5 5 2 2 5 0 3 5 5 4 2 5 4 1 5 5 1 1 1 5 5 1 5
  [217] 3 5 5 3 2 5 3 5 5 5 5 5 5 1 5 2 2 4 5 5 5 5 5 5 3 1 5 5 5 2 4 4 5 5 5 2
  [253] 4 5 5 1 3 5 1 3 5 4 5 5 4 2 5 1 3 4 5 5 5 3 3 2 5 2 5 4 5 3 5 5 5 5 4 0
  [289] 5 4 5 4 5 1 5 2 5 5 4 5 5 5 5 3 5 5 5 1 5 3 5 5 4 5 5 3 4 2 5 5 5 3 5 5
  [325] 5 2 3 3 5 5 3 4 5 2 4 3 1 5 5 5 1 5 3 3 3 3 5 4 5 5 2 4 4 5 5 2 5 5 4 5
  [361] 5 5 2 5 1 1 4 2 2 5 2 1 0 0 2 2 2 2 3 4 3 5 0 2 5 5 0 1 4 2 5 4 5 1 5 5
  [397] 5 2 2 0 0 1 4 4 4 5 5 2 2 3 5 5 4 3 5 4 4 5 5 5 4 5 3 3 5 0 0 5 4 5 4 5
  [433] 5 5 5 2 5 0 4 5 5 3 5 5 3 4 5 4 4 5 5 4 3 5 2 0 5 3 5 5 2 5 5 4 3 3 3 1
  [469] 5 5 5 3 5 3 1 5 5 0 5 2 0 5 2 2 1 5 5 0 5 3 4 5 4 4 1 1 5 4 3 4 5 5 4 0
  [505] 5 2 1 5 5 5 1 1 0 0 5 1 2 0 5 5 1 5 3 0 5 2 0 5 1 4 2 3 5 3 5 5 1 5 2 4
  [541] 5 4 3 3 5 1 5 5 5 3 1 5 4 5 1 1 0 5 0 2 5 5 0 5 4 5 5 2 3 4 1 5 2 2 5 5
  [577] 5 5 5 5 5 5 2 1 5 0 1 5 5 3 5 5 1 2 5 3 5 1 5 4 1 3 4 5 5 5 5 2 1 1 4 5
  [613] 3 5 5 5 3 2 5 5 5 4 3 5 5 5 2 5 5 5 5 5 2 5 4 2 2 5 5 2 2 5 5 5 4 5 3 5
  [649] 4 1 5 5 1 3 5 5 2 5 5 4 2 1 5 5 5 4 5 5 1 3 5 2 2 4 2 5 5 5 0 5 5 5 5 5
  [685] 1 5 5 2 1 5 1 5 2 2 5 2 2 2 3 4 4 1 0 2 0 1 4 5 5 2 1 5 5 5 3 3 5 5 5 2
  [721] 1 2 2 4 5 5 3 2 5 4 5 5 5 5 1 3 4 1 5 2 5 1 2 0 5 5 0 5 0 2 1 3 5 5 5 5
  [757] 5 5 3 2 4 5 5 4 5 5 4 5 2 5 5 5 5 0 1 5 4 1 2 4 5 0 5 5 1 2 1 5 5 3 2 0
  [793] 2 2 5 1 5 3 5 3 5 5 4 5 0 5 3 5 2 5 1 4 5 5 5 5 5 2 5 2 3 3 2 4 1 5 5 5
  [829] 3 2 3 5 3 0 3 5 3 2 1 5 3 2 1 4 2 2 5 5 2 1 1 2 1 5 0 5 3 5 5 2 5 5 5 5
  [865] 4 3 0 2 5 3 4 5 4 1 5 0 1 1 1 3 5 5 5 1 1 5 4 1 5 3 0 4 1 5 5 5 1 4 5 5
  [901] 0 1 0 1 5 5 1 3 4 3 2 2 2 5 1 0 5 3 2 5 2 2 3 5 5 4 2 5 5 1 3 0 2 5 2 2
  [937] 1 5 5 2 5 2 5 4 1 5 5 3 5 5 2 0 0 5 5 5 5 4 4 5 4 1 5 0 4 3 2 5 5 5 3 0
  [973] 2 4 5 5 4 5 2 5 3 5 0 5 3 5 0 4 0 4 2 3 5 3 4 2 3 5 5 5 5 1 1 5 0 5 4 4
 [1009] 1 5 0 5 5 0 5 4 5 5 1 3 2 0 0 2 3 3 2 5 3 5 0 5 5 4 3 5 0 3 5 5 5 5 1 4
 [1045] 0 1 1 1 5 5 5 3 1 3 5 1 5 0 5 4 5 4 5 5 3 5 5 1 3 5 5 5 1 4 5 2 0 5 3 5
 [1081] 1 5 1 5 5 5 0 3 4 4 4 5 4 5 3 2 5 5 4 5 5 5 2 2 5 5 5 5 3 5 5 4 5 3 2 1
 [1117] 0 5 1 5 2 2 3 3 4 2 2 0 5 5 5 5 5 2 5 3 5 1 5 5 3 1 2 5 1 5 4 5 1 4 2 5
 [1153] 3 5 5 5 5 3 5 3 5 5 5 5 5 5 5 1 2 5 3 0 5 1 0 1 5 2 5 4 5 3 3 5 5 2 5 5
 [1189] 2 4 5 2 5 2 5 5 0 5 2 4 5 5 2 5 4 5 5 5 4 5 5 2 5 5 5 2 1 4 5 5 3 5 4 4
 [1225] 4 5 3 1 5 0 5 3 3 0 5 1 5 3 4 5 5 1 4 4 5 5 4 5 5 2 5 3 5 2 4 5 3 2 1 1
 [1261] 5 5 5 5 3 5 4 2 5 5 5 5 1 4 5 5 1 2 5 2 5 5 5 5 1 5 2 2 3 5 2 4 5 1 5 5
 [1297] 4 3 2 3 5 0 0 0 5 5 5 4 5 3 5 5 5 0 5 1 1 2 5 2 5 4 5 5 4 3 2 3 4 4 2 0
 [1333] 5 5 5 2 1 5 5 4 3 5 4 0 5 4 5 0 3 1 0 4 5 1 3 4 3 5 5 1 5 5 5 1 5 4 5 4
 [1369] 4 3 3 5 4 3 4 5 3 4 5 0 3 0 5 5 5 4 5 0 0 2 4 4 5 5 5 0 5 5 5 4 3 4 2 2
 [1405] 5 5 5 0 3 4 3 3 5 2 4 1 5 3 5 5 5 5 5 4 5 2 1 3 5 5 4 5 5 2 5 0 5 2 0 5
 [1441] 5 5 3 1 5 2 5 5 5 5 5 0 5 2 0 5 5 2 5 5 5 5 4 2 1 5 3 5 2 4 2 3 2 5 4 5
 [1477] 5 4 0 5 3 5 0 5 2 5 5 5 5 5 2 5 1 2 5 4 3 3 2 5 5 5 5 4 5 5 5 4 5 5 0 5
 [1513] 5 5 5 5 3 5 2 5 5 1 2 5 5 3 5 5 5 4 2 2 5 3 1 5 5 4 1 5 1 5 1 2 5 1 2 5
 [1549] 3 2 0 5 4 5 5 5 5 5 4 4 3 5 4 3 5 5 5 4 3 4 3 5 5 5 5 5 5 5 3 5 5 5 4 2
 [1585] 5 5 2 1 5 2 5 5 1 3 1 0 3 3 5 2 0 5 2 5 4 5 5 5 5 0 3 5 1 1 2 1 3 2 1 5
 [1621] 3 3 3 5 3 5 5 5 0 3 3 1 2 5 5 5 4 0 3 2 5 5 5 0 4 2 5 5 5 5 5 2 1 1 5 2
 [1657] 5 3 2 5 5 5 5 5 5 5 5 3 4 0 5 0 1 5 5 5 2 5 5 5 2 5 5 5 4 5 3 4 5 3 5 2
 [1693] 3 2 5 1 0 5 2 5 3 4 2 3 5 5 2 5 5 5 5 4 5 2 5 2 2 5 3 4 3 2 2 0 5 1 5 1
 [1729] 5 5 5 5 2 5 5 5 5 5 5 5 2 5 1 5 5 1 5 3 5 5 5 2 5 5 2 2 5 0 4 3 5 4 5 3
 [1765] 5 5 5 5 3 0 5 3 5 5 5 1 5 3 4 5 5 5 0 1 5 1 5 5 5 1 3 5 2 5 3 3 5 1 2 1
 [18

In [46]:
cor.test(NIJ$Prior_Arrest_Episodes_PPViolationCharges, NIJ$Race, method='spearman', use='complete.obs')

Warning message in cor.test.default(NIJ$Prior_Arrest_Episodes_PPViolationCharges, :
“Cannot compute exact p-value with ties”



	Spearman's rank correlation rho

data:  NIJ$Prior_Arrest_Episodes_PPViolationCharges and NIJ$Race
S = 2.7042e+12, p-value < 2.2e-16
alternative hypothesis: true rho is not equal to 0
sample estimates:
      rho 
0.0590605 


In [22]:
cor.test(NIJ$Percent_Days_Employed, NIJ$Avg_Days_per_DrugTest)



	Pearson's product-moment correlation

data:  NIJ$Percent_Days_Employed and NIJ$Avg_Days_per_DrugTest
t = -12.28, df = 19730, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.10092205 -0.07322769
sample estimates:
       cor 
-0.0870917 


In [47]:
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '18-22', '18')
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '23-27', '23')
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '28-32', '28')
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '33-37', '33')
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '38-42', '38')
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '43-47', '43')
NIJ$Age_at_Release <- replace(NIJ$Age_at_Release, NIJ$Age_at_Release == '48 or older', '48')
NIJ$Age_at_Release <-  as.integer(NIJ$Age_at_Release)
View(NIJ$Age_at_Release)

[1] 43 33 48 38 33 38 48 38 43 43 43 33 48 33 33 33 38 43 48 48 38 33 48 43
   [25] 38 48 48 48 43 18 43 43 43 38 33 48 43 33 33 38 33 48 43 33 43 38 38 38
   [49] 33 38 43 33 18 48 38 23 38 33 38 33 43 33 48 43 43 48 33 28 33 33 48 38
   [73] 33 33 38 43 43 43 33 33 33 33 48 38 33 43 28 33 33 43 33 38 33 28 33 43
   [97] 28 33 48 48 33 48 38 33 43 43 38 48 43 48 33 38 33 43 38 48 48 48 48 48
  [121] 38 33 28 33 48 33 38 48 33 33 43 33 48 33 38 38 33 33 33 38 43 33 48 48
  [145] 43 38 33 38 28 28 33 33 48 48 38 33 48 48 33 38 43 43 48 33 28 43 33 48
  [169] 28 48 38 38 28 38 48 33 38 23 38 43 28 48 28 28 33 33 33 28 33 38 28 38
  [193] 33 48 33 43 33 38 43 23 43 33 38 48 48 28 48 48 33 33 33 38 43 48 43 43
  [217] 38 48 48 33 43 48 48 43 28 33 48 28 38 33 38 48 33 38 38 43 38 43 38 28
  [241] 33 28 43 43 33 28 43 48 43 43 38 28 38 48 33 33 43 33 28 28 43 28 33 48
  [265] 48 43 48 33 48 28 38 33 28 28 43 28 28 38 28 48 43 48 48 28 33 33 48 38
  [289] 33 28 48 28 38 33 33 28 33 48 43 28 28 28 43 43 43 48 33 28 38 33 48 43
  [313] 28 33 33 48 33 28 48 38 33 28 43 28 43 33 43 28 43 43 33 48 48 33 48 38
  [337] 23 43 33 28 43 43 33 38 48 33 38 28 43 28 48 33 33 48 38 33 43 43 38 33
  [361] 43 33 33 38 33 28 43 48 48 33 28 38 43 43 33 33 48 28 33 43 33 33 43 28
  [385] 38 33 33 28 43 28 33 33 48 28 28 33 38 48 43 33 38 48 38 48 33 28 48 38
  [409] 48 38 38 33 48 43 28 48 48 33 48 48 48 43 48 33 38 18 43 38 33 48 48 33
  [433] 33 28 48 33 43 33 33 28 43 33 43 48 48 33 48 33 28 48 28 28 33 43 28 33
  [457] 38 43 43 43 28 33 43 48 38 33 43 33 28 33 48 33 48 33 38 33 33 48 43 33
  [481] 33 48 48 38 33 33 33 43 33 33 38 43 38 28 38 33 28 38 33 38 38 33 48 28
  [505] 33 33 18 38 43 48 48 33 38 48 38 28 38 43 38 28 28 48 28 33 48 33 48 23
  [529] 43 33 38 33 28 48 33 38 43 28 33 43 33 48 38 48 28 43 28 38 38 28 48 48
  [553] 38 28 48 38 43 48 38 33 33 48 38 38 28 33 33 33 48 33 28 48 28 28 28 38
  [577] 28 38 38 38 43 33 33 33 33 38 48 48 33 38 28 48 28 38 43 38 43 38 28 48
  [601] 38 43 33 33 48 33 43 48 43 38 43 28 48 38 28 33 28 48 48 33 28 38 48 28
  [625] 33 28 33 38 33 43 33 48 33 43 48 28 33 43 48 28 38 28 28 38 43 33 48 28
  [649] 28 48 43 33 38 33 43 43 28 28 28 38 33 48 33 38 28 33 28 33 28 38 28 43
  [673] 33 33 28 48 33 48 33 33 43 33 33 33 38 38 33 33 38 33 43 33 48 48 48 33
  [697] 43 33 33 33 33 33 28 48 33 48 48 33 33 38 33 33 28 38 48 48 48 33 33 38
  [721] 48 28 28 28 43 43 28 48 43 43 48 48 48 33 33 28 33 28 28 33 43 38 28 43
  [745] 33 28 38 28 28 38 33 33 33 48 43 33 33 48 48 23 38 48 33 33 48 43 33 28
  [769] 33 33 48 43 33 33 33 33 33 33 38 28 33 28 43 33 33 43 38 33 33 28 43 38
  [793] 38 28 38 33 43 38 33 33 43 28 43 33 43 38 28 48 38 28 43 33 33 28 33 28
  [817] 33 48 48 48 28 33 33 33 48 43 48 48 48 33 38 48 38 43 28 38 33 48 38 48
  [841] 48 38 43 38 33 43 48 38 43 33 38 38 28 28 33 48 48 38 43 48 38 48 48 33
  [865] 48 38 48 33 33 33 43 43 33 48 43 38 38 33 33 33 38 38 43 33 38 28 48 38
  [889] 43 48 48 48 33 38 43 28 33 33 43 28 38 33 33 28 38 48 48 38 33 28 43 43
  [913] 43 33 33 48 33 33 43 43 28 48 43 33 48 33 28 43 33 48 28 38 28 33 33 48
  [937] 43 33 28 33 48 48 48 33 43 33 33 33 48 48 33 38 33 33 33 43 33 38 33 33
  [961] 33 33 38 48 33 38 43 38 33 38 33 38 33 33 33 38 33 38 33 48 28 38 43 38
  [985] 28 38 43 28 38 43 43 48 33 38 43 33 28 48 48 28 33 48 38 43 33 43 33 33
 [1009] 48 48 38 28 33 33 38 33 38 48 28 38 33 33 43 43 48 33 48 43 28 28 33 43
 [1033] 48 43 33 43 48 33 38 33 38 33 33 33 43 48 38 33 48 48 33 48 33 33 33 38
 [1057] 28 38 38 48 38 33 33 48 33 48 38 33 33 33 28 48 33 28 38 28 28 43 33 48
 [1081] 28 33 38 43 38 38 43 48 28 28 38 38 48 48 28 33 43 33 33 48 33 33 28 38
 [1105] 33 33 38 43 43 38 33 38 33 28 28 33 43 48 33 33 48 48 33 33 28 33 38 33
 [1129] 33 33 33 43 28 33 28 28 43 43 33 38 28 38 43 38 28 43 38 33 33 33 33 33
 [1153] 28 28 33 48 43 48 28 38 33 38 43 33 38 43 33 38 33 33 48 48 43 48 48 33
 [1177] 43 43 48 38 28 28 43 48 38 38 43 38 43 43 38 48 33 48 33 28 43 38 33 43
 [12

In [49]:
cor.test(NIJ$Age_at_Release, NIJ$Race, method='spearman', use='complete.obs')

Warning message in cor.test.default(NIJ$Age_at_Release, NIJ$Race, method = "spearman", :
“Cannot compute exact p-value with ties”



	Spearman's rank correlation rho

data:  NIJ$Age_at_Release and NIJ$Race
S = 2.515e+12, p-value < 2.2e-16
alternative hypothesis: true rho is not equal to 0
sample estimates:
      rho 
0.1249048 


Is there a statisitically significant difference in felony prior arest episodes betweeen black males and white males?

Is there a statisitically significant difference in property prior arest episodes betweeen black males and white males?

Is there a statisitically significant difference in supervision risk score betweeen black males and white males?

Is there a statisitically significant difference in misdemeanour prior arrest episodes betweeen black males and white males?

Is there a statisitically significant difference in misdemeanour prior conviction episodes betweeen black males and white males?

Is there a statisitically significant difference in property prior conviction episodes betweeen black males and white males?

Is there a statisitically significant difference in percentage of days employed betweeen black males and white males?


Is there a statisitically significant difference in age betweeen black males and white males?

Is there a statisitically significant difference in positive Meth drug tests betweeen black males and white males?

Is there a statisitically significant difference in residence areas betweeen black males and white males?

Is there a statisitically significant difference in positive THC drug tests betweeen black men and white men?

Is there a statisitically significant difference in positive cocaine drug tests betweeen black males and white males?

Is there a statisitically significant difference in violent non-sexual offences betweeen black males and white males?

Is there a statisitically significant difference in violence prior arrest episodes betweeen black males and white males?

Is there a statisitically significant difference in mental health or substance abuse conditions betweeen black men and white men?

Is there a statisitically significant difference in percentage of days employed betweeen black males and white males?

Is there a statisitically significant difference in jobs per year betweeen black males and white males?

Is there a statisitically significant difference in gang affiliation betweeen black males and white males?

Is there a statisitically significant difference in violence prior arrest episodes betweeen black females and white females?